In [2]:
# M1 model for a particular DMU

import pulp
import numpy as np
import pandas as pd

# Example data: input and output matrices (replace with actual dataset)
# Assuming we have 5 DMUs, 3 inputs, and 2 outputs.
inputs = np.array([
    [50, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])

# Parameters
n_dmus = inputs.shape[0]  # Number of DMUs
n_inputs = inputs.shape[1]  # Number of inputs
n_outputs = outputs.shape[1]  # Number of outputs
epsilon = 1e-6  # Small positive number for slack penalty

# Select the DMU to optimize (for example, DMU 0)
dmu_0 = 0
i_0 = 0  # Index of the input we want to prioritize minimizing

# Create a linear programming problem
prob = pulp.LpProblem("M1_Model", pulp.LpMinimize)

# Decision variables
h_0 = pulp.LpVariable("h_0", lowBound=None)  # h_0 is free to be > or < 0
lambda_vars = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(n_dmus)]
s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}", lowBound=0) for i in range(n_inputs) if i != i_0]
s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}", lowBound=0) for r in range(n_outputs)]

# Objective function: Minimize h_0 - epsilon * (sum(s_i_minus) + sum(s_r_plus))
prob += h_0 - epsilon * (pulp.lpSum(s_i_minus) + pulp.lpSum(s_r_plus)), "Objective"

# Constraints

# Input constraint for prioritized input i_0
prob += h_0 * inputs[dmu_0, i_0] == pulp.lpSum(lambda_vars[j] * inputs[j, i_0] for j in range(n_dmus))

# Input constraints for all other inputs (i != i_0)
for i in range(n_inputs):
    if i != i_0:
        prob += pulp.lpSum(lambda_vars[j] * inputs[j, i] for j in range(n_dmus)) + s_i_minus[i - 1] == inputs[dmu_0, i]

# Output constraints
for r in range(n_outputs):
    prob += pulp.lpSum(lambda_vars[j] * outputs[j, r] for j in range(n_dmus)) - s_r_plus[r] == outputs[dmu_0, r]

# Solve the problem
prob.solve()

# Output results
print("Status:", pulp.LpStatus[prob.status])
print(f"Optimal value of h_0: {h_0.varValue}")

for i in range(n_inputs):
    if i != i_0:
        print(f"s_i_minus_{i}: {s_i_minus[i - 1].varValue}")

for r in range(n_outputs):
    print(f"s_r_plus_{r}: {s_r_plus[r].varValue}")

for j in range(n_dmus):
    print(f"lambda_{j}: {lambda_vars[j].varValue}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/754932d6607a4b6d8a857d494044d307-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/754932d6607a4b6d8a857d494044d307-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 46 RHS
At line 52 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-1) rows, 9 (-1) columns and 24 (-6) elements
0  Obj 0 Primal inf 3.1745992 (4) Dual inf 2.6e-05 (4)
4  Obj 1
Optimal - objective value 1
After Postsolve, objective 1, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1 - 4 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from n

In [6]:
# M1 model performimg operation for each DMU

import pulp
import numpy as np
import pandas as pd

# Example data: input and output matrices (replace with actual dataset)
# Assuming we have 5 DMUs, 3 inputs, and 2 outputs.
inputs = np.array([
    [3, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])

# Parameters
n_dmus = inputs.shape[0]  # Number of DMUs
n_inputs = inputs.shape[1]  # Number of inputs
n_outputs = outputs.shape[1]  # Number of outputs
epsilon = 1e-8  # Small positive number for slack penalty

# Results storage
results = {
    "DMU": [],
    "h_0": [],
    "lambda_values": [],
    "s_i_minus": [],
    "s_r_plus": []
}

# Loop through each DMU and solve the M1 model
for dmu_0 in range(n_dmus):
    # Create a linear programming problem for each DMU
    prob = pulp.LpProblem(f"M1_Model_DMU_{dmu_0}", pulp.LpMinimize)

    # Decision variables
    h_0 = pulp.LpVariable(f"h_0_{dmu_0}", lowBound=None)  # h_0 is free to be > or < 0
    lambda_vars = [pulp.LpVariable(f"lambda_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) for i in range(1, n_inputs)]
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) for r in range(n_outputs)]

    # Objective function: Minimize h_0 - epsilon * (sum(s_i_minus) + sum(s_r_plus))
    prob += h_0 - epsilon * (pulp.lpSum(s_i_minus) + pulp.lpSum(s_r_plus)), f"Objective_DMU_{dmu_0}"

    # Input constraint for prioritized input i_0 (first input is prioritized)
    prob += h_0 * inputs[dmu_0, 0] == pulp.lpSum(lambda_vars[j] * inputs[j, 0] for j in range(n_dmus))

    # Input constraints for all other inputs (i != i_0)
    for i in range(1, n_inputs):
        prob += pulp.lpSum(lambda_vars[j] * inputs[j, i] for j in range(n_dmus)) + s_i_minus[i - 1] == inputs[dmu_0, i]

    # Output constraints
    for r in range(n_outputs):
        prob += pulp.lpSum(lambda_vars[j] * outputs[j, r] for j in range(n_dmus)) - s_r_plus[r] == outputs[dmu_0, r]

    # Solve the problem
    prob.solve()

    # Store results for this DMU
    lambda_values = [lambda_vars[j].varValue for j in range(n_dmus)]
    s_i_minus_values = [s_i_minus[i].varValue for i in range(n_inputs - 1)]
    s_r_plus_values = [s_r_plus[r].varValue for r in range(n_outputs)]

    results["DMU"].append(dmu_0)
    results["h_0"].append(h_0.varValue)
    results["lambda_values"].append(lambda_values)
    results["s_i_minus"].append(s_i_minus_values)
    results["s_r_plus"].append(s_r_plus_values)

# Convert results to a DataFrame for better readability
df_results = pd.DataFrame(results)

# Display the results for all DMUs
print(df_results)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/e844e65810bc4c0db53e2e4fc9657754-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/e844e65810bc4c0db53e2e4fc9657754-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 46 RHS
At line 52 BOUNDS
At line 54 ENDATA
Problem MODEL has 5 rows, 10 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-1) rows, 9 (-1) columns and 24 (-6) elements
0  Obj 0 Primal inf 3.1745992 (4)
2  Obj 1
Optimal - objective value 1
After Postsolve, objective 1, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total ti

In [7]:
# M1 model performimg operation for each DMU

import pulp
import numpy as np
import pandas as pd

# Example data: input and output matrices (replace with actual dataset)
# Assuming we have 4 DMUs, 2 inputs, and 3 outputs.
inputs = np.array([
    [10, 15 ],
    [10, 7 ],
    [12, 9 ],
    [4, 16 ]
])

outputs = np.array([
    [8, 12 , 3],
    [12, 8 , 5],
    [10, 10 , 4],
    [4, 15 , 0.5]
])

# Parameters
n_dmus = inputs.shape[0]  # Number of DMUs
n_inputs = inputs.shape[1]  # Number of inputs
n_outputs = outputs.shape[1]  # Number of outputs
epsilon = 1e-8  # Small positive number for slack penalty

# Results storage
results = {
    "DMU": [],
    "h_0": [],
    "lambda_values": [],
    "s_i_minus": [],
    "s_r_plus": []
}

# Loop through each DMU and solve the M1 model
for dmu_0 in range(n_dmus):
    # Create a linear programming problem for each DMU
    prob = pulp.LpProblem(f"M1_Model_DMU_{dmu_0}", pulp.LpMinimize)

    # Decision variables
    h_0 = pulp.LpVariable(f"h_0_{dmu_0}", lowBound=None)  # h_0 is free to be > or < 0
    lambda_vars = [pulp.LpVariable(f"lambda_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) for i in range(1, n_inputs)]
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) for r in range(n_outputs)]

    # Objective function: Minimize h_0 - epsilon * (sum(s_i_minus) + sum(s_r_plus))
    prob += h_0 - epsilon * (pulp.lpSum(s_i_minus) + pulp.lpSum(s_r_plus)), f"Objective_DMU_{dmu_0}"

    # Input constraint for prioritized input i_0 (first input is prioritized)
    prob += h_0 * inputs[dmu_0, 0] == pulp.lpSum(lambda_vars[j] * inputs[j, 0] for j in range(n_dmus))

    # Input constraints for all other inputs (i != i_0)
    for i in range(1, n_inputs):
        prob += pulp.lpSum(lambda_vars[j] * inputs[j, i] for j in range(n_dmus)) + s_i_minus[i - 1] == inputs[dmu_0, i]

    # Output constraints
    for r in range(n_outputs):
        prob += pulp.lpSum(lambda_vars[j] * outputs[j, r] for j in range(n_dmus)) - s_r_plus[r] == outputs[dmu_0, r]

    # Solve the problem
    prob.solve()

    # Store results for this DMU
    lambda_values = [lambda_vars[j].varValue for j in range(n_dmus)]
    s_i_minus_values = [s_i_minus[i].varValue for i in range(n_inputs - 1)]
    s_r_plus_values = [s_r_plus[r].varValue for r in range(n_outputs)]

    results["DMU"].append(dmu_0)
    results["h_0"].append(h_0.varValue)
    results["lambda_values"].append(lambda_values)
    results["s_i_minus"].append(s_i_minus_values)
    results["s_r_plus"].append(s_r_plus_values)

# Convert results to a DataFrame for better readability
df_results = pd.DataFrame(results)

# Display the results for all DMUs
print(df_results)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/a93798e4d3a84dacb06ab410d7c31874-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/a93798e4d3a84dacb06ab410d7c31874-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 49 ENDATA
Problem MODEL has 5 rows, 9 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-1) rows, 8 (-1) columns and 20 (-5) elements
0  Obj 0 Primal inf 9.4313114 (4)
4  Obj 0.75211268
Optimal - objective value 0.75211264
After Postsolve, objective 0.75211264, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.7521126394 - 4 iterations time 0.002, Presolve 0.00
Option for printingOption

In [9]:
# Fixed M1 Model , fixing high slack variables 

import pulp
import numpy as np
import pandas as pd

# Example data: input and output matrices (replace with actual dataset)
inputs = np.array([
    [10, 15 ],
    [10, 7 ],
    [12, 9 ],
    [4, 16 ]
])

outputs = np.array([
    [8, 12 , 3],
    [12, 8 , 5],
    [10, 10 , 4],
    [4, 15 , 0.5]
])

# Normalize the inputs and outputs (optional but recommended if values differ widely)
inputs_normalized = inputs / np.max(inputs, axis=0)
outputs_normalized = outputs / np.max(outputs, axis=0)

# Parameters
n_dmus = inputs.shape[0]  # Number of DMUs
n_inputs = inputs.shape[1]  # Number of inputs
n_outputs = outputs.shape[1]  # Number of outputs
epsilon = 1e-4  # Increase epsilon to penalize slack variables more

# Results storage
results = {
    "DMU": [],
    "h_0": [],
    "lambda_values": [],
    "s_i_minus": [],
    "s_r_plus": []
}

# Loop through each DMU and solve the M1 model
for dmu_0 in range(n_dmus):
    # Create a linear programming problem for each DMU
    prob = pulp.LpProblem(f"M1_Model_DMU_{dmu_0}", pulp.LpMinimize)

    # Decision variables
    h_0 = pulp.LpVariable(f"h_0_{dmu_0}", lowBound=None)  # h_0 is free to be > or < 0
    lambda_vars = [pulp.LpVariable(f"lambda_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]  # Ensure lambda <= 1
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) for i in range(1, n_inputs)]
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) for r in range(n_outputs)]

    # Objective function: Minimize h_0 - epsilon * (sum(s_i_minus) + sum(s_r_plus))
    prob += h_0 - epsilon * (pulp.lpSum(s_i_minus) + pulp.lpSum(s_r_plus)), f"Objective_DMU_{dmu_0}"

    # Input constraint for prioritized input i_0 (first input is prioritized)
    prob += h_0 * inputs_normalized[dmu_0, 0] == pulp.lpSum(lambda_vars[j] * inputs_normalized[j, 0] for j in range(n_dmus))

    # Input constraints for all other inputs (i != i_0)
    for i in range(1, n_inputs):
        prob += pulp.lpSum(lambda_vars[j] * inputs_normalized[j, i] for j in range(n_dmus)) + s_i_minus[i - 1] == inputs_normalized[dmu_0, i]

    # Output constraints
    for r in range(n_outputs):
        prob += pulp.lpSum(lambda_vars[j] * outputs_normalized[j, r] for j in range(n_dmus)) - s_r_plus[r] == outputs_normalized[dmu_0, r]

    # Solve the problem
    prob.solve()

    # Store results for this DMU
    lambda_values = [lambda_vars[j].varValue for j in range(n_dmus)]
    s_i_minus_values = [s_i_minus[i].varValue for i in range(n_inputs - 1)]
    s_r_plus_values = [s_r_plus[r].varValue for r in range(n_outputs)]

    results["DMU"].append(dmu_0)
    results["h_0"].append(h_0.varValue)
    results["lambda_values"].append(lambda_values)
    results["s_i_minus"].append(s_i_minus_values)
    results["s_r_plus"].append(s_r_plus_values)

# Convert results to a DataFrame for better readability
df_results = pd.DataFrame(results)

# Display the results for all DMUs
print(df_results)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/8a6fba630776443f827c25c78412dfea-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/8a6fba630776443f827c25c78412dfea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 49 ENDATA
Problem MODEL has 5 rows, 9 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-1) rows, 8 (-1) columns and 20 (-5) elements
0  Obj 0 Primal inf 5.5891698 (4) Dual inf 0.00022334155 (4)
5  Obj 0.7520885
Optimal - objective value 0.7520885
After Postsolve, objective 0.7520885, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.7520884977 - 5 iterations time 0.002, Presolve 0.00
O

In [10]:
import pulp
import numpy as np
import pandas as pd

# Example input and output data
inputs = np.array([
    [3, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])

# Number of DMUs, inputs, and outputs
n_dmus = inputs.shape[0]
n_inputs = inputs.shape[1]
n_outputs = outputs.shape[1]

# Assign weights to specific inputs and outputs
weighted_inputs = [0, 2]  # Input indices with weights
weighted_outputs = [1]    # Output indices with weights

# Weight values for weighted inputs and outputs
input_weights = np.array([2, 3])  # Weights for inputs with indices [0, 2]
output_weights = np.array([1.5])  # Weight for output with index [1]

# Small epsilon for slack penalties
epsilon = 1e-6

# Store results for each DMU
all_results = []

# Loop through each DMU and solve the M3 model
for dmu_0 in range(n_dmus):
    # Create an LP problem for the DMU
    prob = pulp.LpProblem(f"M3_Model_DMU_{dmu_0}", pulp.LpMaximize)

    # Decision variables for scaling inputs (p_i) and outputs (z_r)
    p_i = [pulp.LpVariable(f"p_{i}_DMU_{dmu_0}", lowBound=0, upBound=1) if i in weighted_inputs else None 
           for i in range(n_inputs)]
    z_r = [pulp.LpVariable(f"z_{r}_DMU_{dmu_0}", lowBound=1) if r in weighted_outputs else None 
           for r in range(n_outputs)]

    # Slack variables for unweighted inputs and outputs
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) if i not in weighted_inputs else None 
                 for i in range(n_inputs)]
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) if r not in weighted_outputs else None 
                for r in range(n_outputs)]

    # Peer weights (theta) for each DMU
    theta = [pulp.LpVariable(f"theta_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]

    # Objective function: Maximize weighted output increases - weighted input reductions + slack penalties
    prob += (
        pulp.lpSum([output_weights[weighted_outputs.index(r)] * z_r[r] for r in weighted_outputs]) -
        pulp.lpSum([input_weights[weighted_inputs.index(i)] * p_i[i] for i in weighted_inputs]) +
        epsilon * (
            pulp.lpSum([s for s in s_i_minus if s is not None]) + 
            pulp.lpSum([s for s in s_r_plus if s is not None])
        )
    ), f"Objective_DMU_{dmu_0}"

    # Constraints for weighted outputs
    for r in weighted_outputs:
        prob += z_r[r] * outputs[dmu_0, r] == pulp.lpSum(
            theta[j] * outputs[j, r] for j in range(n_dmus)
        ), f"Weighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Constraints for weighted inputs
    for i in weighted_inputs:
        prob += p_i[i] * inputs[dmu_0, i] == pulp.lpSum(
            theta[j] * inputs[j, i] for j in range(n_dmus)
        ), f"Weighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted inputs with slack variables
    for i in range(n_inputs):
        if i not in weighted_inputs:
            prob += pulp.lpSum([theta[j] * inputs[j, i] for j in range(n_dmus)]) + s_i_minus[i] == inputs[dmu_0, i], \
                    f"Unweighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted outputs with slack variables
    for r in range(n_outputs):
        if r not in weighted_outputs:
            prob += pulp.lpSum([theta[j] * outputs[j, r] for j in range(n_dmus)]) - s_r_plus[r] == outputs[dmu_0, r], \
                    f"Unweighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Convexity constraint: Sum of theta_j = 1
    prob += pulp.lpSum(theta) == 1, f"Convexity_Constraint_DMU_{dmu_0}"

    # Solve the LP problem for the current DMU
    prob.solve()

    # Extract the results for the DMU
    p_i_values = [pulp.value(p) if p is not None else None for p in p_i]
    z_r_values = [pulp.value(z) if z is not None else None for z in z_r]
    theta_values = [pulp.value(t) for t in theta]
    s_i_minus_values = [pulp.value(s) if s is not None else None for s in s_i_minus]
    s_r_plus_values = [pulp.value(s) if s is not None else None for s in s_r_plus]

    # Store results in a structured way
    result = {
        "DMU": dmu_0,
        "p_i": p_i_values,
        "z_r": z_r_values,
        "theta": theta_values,
        "s_i_minus": s_i_minus_values,
        "s_r_plus": s_r_plus_values
    }
    all_results.append(result)

# Convert results to a DataFrame and display them
df_results = pd.DataFrame(all_results)
print(df_results)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/6bf050b30b1c4d19996b6736b39c789b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/6bf050b30b1c4d19996b6736b39c789b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 63 ENDATA
Problem MODEL has 6 rows, 10 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 10 (0) columns and 35 (0) elements
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.5000157 (3)
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.7e+11 (6)
7  Obj -3.5
Optimal - objective value -3.5
Optimal objective -3.5 - 7 iterations time 0.002
Option for printingOptions changed from normal to all
To

In [16]:
import pulp
import numpy as np
import pandas as pd

# Example input and output data
inputs = np.array([
    [3, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])

# Number of DMUs, inputs, and outputs
n_dmus = inputs.shape[0]
n_inputs = inputs.shape[1]
n_outputs = outputs.shape[1]

# Assign weights to specific inputs and outputs
weighted_inputs = [0, 2]  # Input indices with weights
weighted_outputs = [1]    # Output indices with weights

# Weight values for weighted inputs and outputs
input_weights = np.array([2, 3])  # Weights for inputs with indices [0, 2]
output_weights = np.array([1.5])  # Weight for output with index [1]

# Small epsilon for slack penalties
epsilon = 1e-6

# Store results for each DMU
all_results = []

# Loop through each DMU and solve the M3 model
for dmu_0 in range(n_dmus):
    # Create an LP problem for the DMU
    prob = pulp.LpProblem(f"M3_Model_DMU_{dmu_0}", pulp.LpMaximize)

    # Decision variables for scaling inputs (p_i) and outputs (z_r)
    p_i = [pulp.LpVariable(f"p_{i}_DMU_{dmu_0}", lowBound=0, upBound=1) if i in weighted_inputs else None 
           for i in range(n_inputs)]
    z_r = [pulp.LpVariable(f"z_{r}_DMU_{dmu_0}", lowBound=1) if r in weighted_outputs else None 
           for r in range(n_outputs)]

    # Slack variables for unweighted inputs and outputs
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) if i not in weighted_inputs else None 
                 for i in range(n_inputs)]
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) if r not in weighted_outputs else None 
                for r in range(n_outputs)]

    # Peer weights (theta) for each DMU
    theta = [pulp.LpVariable(f"theta_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]

    # Objective function: Maximize weighted output increases - weighted input reductions + slack penalties
    prob += (
        pulp.lpSum([output_weights[weighted_outputs.index(r)] * z_r[r] for r in weighted_outputs]) -
        pulp.lpSum([input_weights[weighted_inputs.index(i)] * p_i[i] for i in weighted_inputs]) +
        epsilon * (
            pulp.lpSum([s for s in s_i_minus if s is not None]) + 
            pulp.lpSum([s for s in s_r_plus if s is not None])
        )
    ), f"Objective_DMU_{dmu_0}"

    # Constraints for weighted outputs
    for r in weighted_outputs:
        prob += z_r[r] * outputs[dmu_0, r] == pulp.lpSum(
            theta[j] * outputs[j, r] for j in range(n_dmus)
        ), f"Weighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Constraints for weighted inputs
    for i in weighted_inputs:
        prob += p_i[i] * inputs[dmu_0, i] == pulp.lpSum(
            theta[j] * inputs[j, i] for j in range(n_dmus)
        ), f"Weighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted inputs with slack variables
    for i in range(n_inputs):
        if i not in weighted_inputs:
            prob += pulp.lpSum([theta[j] * inputs[j, i] for j in range(n_dmus)]) + s_i_minus[i] == inputs[dmu_0, i], \
                    f"Unweighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted outputs with slack variables
    for r in range(n_outputs):
        if r not in weighted_outputs:
            prob += pulp.lpSum([theta[j] * outputs[j, r] for j in range(n_dmus)]) - s_r_plus[r] == outputs[dmu_0, r], \
                    f"Unweighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Convexity constraint: Sum of theta_j = 1
    prob += pulp.lpSum(theta) == 1, f"Convexity_Constraint_DMU_{dmu_0}"

    # Solve the LP problem for the current DMU
    prob.solve()

    # Extract the results for the DMU
    p_i_values = [pulp.value(p) if p is not None else None for p in p_i]
    z_r_values = [pulp.value(z) if z is not None else None for z in z_r]
    theta_values = [pulp.value(t) for t in theta]
    s_i_minus_values = [pulp.value(s) if s is not None else None for s in s_i_minus]
    s_r_plus_values = [pulp.value(s) if s is not None else None for s in s_r_plus]

    # Store results in a structured way
    result = {
        "DMU": dmu_0,
        "p_i": p_i_values,
        "z_r": z_r_values,
        "theta": theta_values,
        "s_i_minus": s_i_minus_values,
        "s_r_plus": s_r_plus_values
    }
    all_results.append(result)

# Convert results to a DataFrame and display them
df_results = pd.DataFrame(all_results)
print(df_results)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/90740d86c4a348ee9da28f96fb454b55-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/90740d86c4a348ee9da28f96fb454b55-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 63 ENDATA
Problem MODEL has 6 rows, 10 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 10 (0) columns and 35 (0) elements
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.5000157 (3)
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.7e+11 (6)
8  Obj -30000.5
Optimal - objective value -30000.5
Optimal objective -30000.5 - 8 iterations time 0.002
Option for printingOptions changed from norm

In [11]:
# M3 code with modified to take values from csv dataset and also sum of theta = 1
import pandas as pd
import numpy as np
import pulp

# Load dataset with DMUs, inputs, and outputs (assumed CSV format)
df = pd.read_csv("Untitled spreadsheet - Sheet1.csv")

# Extract DMUs, inputs, and outputs from the dataset
dmu_names = df.iloc[:-1, 0]  # All DMUs (excluding last row)
inputs = df.iloc[:-1, 1:4].values.astype(float)  # Inputs (excluding last row)
outputs = df.iloc[:-1, 4:].values.astype(float)  # Outputs (excluding last row)

# Extract weights from the last row (if 'None', set as unweighted)
input_weights = df.iloc[-1, 1:4].replace("None", np.nan).astype(float).values
output_weights = df.iloc[-1, 4:].replace("None", np.nan).astype(float).values

# Get the number of DMUs, inputs, and outputs
n_dmus = inputs.shape[0]
n_inputs = inputs.shape[1]
n_outputs = outputs.shape[1]

# Store results for each DMU
all_results = []

# Loop through each DMU and solve the M3 model
for dmu_0 in range(n_dmus):
    # Create an LP problem for the DMU
    prob = pulp.LpProblem(f"M3_Model_DMU_{dmu_0}", pulp.LpMaximize)

    # Decision variables for scaling inputs and outputs
    p_i = [pulp.LpVariable(f"p_{i}_DMU_{dmu_0}", lowBound=0, upBound=1) 
           if not np.isnan(input_weights[i]) else None for i in range(n_inputs)]
    
    z_r = [pulp.LpVariable(f"z_{r}_DMU_{dmu_0}", lowBound=1) 
           if not np.isnan(output_weights[r]) else None for r in range(n_outputs)]

    # Slack variables for unweighted inputs and outputs
    s_i_minus = [pulp.LpVariable(f"s_i_minus_{i}_DMU_{dmu_0}", lowBound=0) 
                 if np.isnan(input_weights[i]) else None for i in range(n_inputs)]
    
    s_r_plus = [pulp.LpVariable(f"s_r_plus_{r}_DMU_{dmu_0}", lowBound=0) 
                if np.isnan(output_weights[r]) else None for r in range(n_outputs)]

    # Peer weights (theta) for each DMU
    theta = [pulp.LpVariable(f"theta_{j}_DMU_{dmu_0}", lowBound=0) for j in range(n_dmus)]

    # Objective function
    prob += (
        pulp.lpSum([output_weights[r] * z_r[r] for r in range(n_outputs) if z_r[r] is not None]) -
        pulp.lpSum([input_weights[i] * p_i[i] for i in range(n_inputs) if p_i[i] is not None]) +
        1e-6 * (pulp.lpSum([s for s in s_i_minus if s is not None]) + 
                pulp.lpSum([s for s in s_r_plus if s is not None]))
    ), f"Objective_DMU_{dmu_0}"

    # Constraints for weighted outputs
    for r in range(n_outputs):
        if z_r[r] is not None:
            prob += z_r[r] * outputs[dmu_0, r] == pulp.lpSum(
                theta[j] * outputs[j, r] for j in range(n_dmus)
            ), f"Weighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Constraints for weighted inputs
    for i in range(n_inputs):
        if p_i[i] is not None:
            prob += p_i[i] * inputs[dmu_0, i] == pulp.lpSum(
                theta[j] * inputs[j, i] for j in range(n_dmus)
            ), f"Weighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted inputs with slack variables
    for i in range(n_inputs):
        if s_i_minus[i] is not None:
            prob += pulp.lpSum([theta[j] * inputs[j, i] for j in range(n_dmus)]) + s_i_minus[i] == inputs[dmu_0, i], \
                    f"Unweighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Constraints for unweighted outputs with slack variables
    for r in range(n_outputs):
        if s_r_plus[r] is not None:
            prob += pulp.lpSum([theta[j] * outputs[j, r] for j in range(n_dmus)]) - s_r_plus[r] == outputs[dmu_0, r], \
                    f"Unweighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Convexity constraint: Sum of theta_j = 1
    prob += pulp.lpSum(theta) == 1, f"Convexity_Constraint_DMU_{dmu_0}"

    # Solve the LP problem for the current DMU
    prob.solve()

    # Extract results for the DMU
    p_i_values = [pulp.value(p) if p is not None else None for p in p_i]
    z_r_values = [pulp.value(z) if z is not None else None for z in z_r]
    theta_values = [pulp.value(t) for t in theta]
    s_i_minus_values = [pulp.value(s) if s is not None else None for s in s_i_minus]
    s_r_plus_values = [pulp.value(s) if s is not None else None for s in s_r_plus]

    # Store results in a structured way
    result = {
        "DMU": dmu_names.iloc[dmu_0],
        "p_i": p_i_values,
        "z_r": z_r_values,
        "theta": theta_values,
        "s_i_minus": s_i_minus_values,
        "s_r_plus": s_r_plus_values
    }
    all_results.append(result)

# Convert results to a DataFrame and display them
df_results = pd.DataFrame(all_results)
print(df_results)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/9a765904bf4e4e57b3ad8b362bb5778b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/9a765904bf4e4e57b3ad8b362bb5778b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 52 RHS
At line 59 BOUNDS
At line 63 ENDATA
Problem MODEL has 6 rows, 10 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 6 (0) rows, 10 (0) columns and 35 (0) elements
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.5000157 (3)
0  Obj 1.5 Primal inf 3.1746028 (4) Dual inf 1.7e+11 (6)
7  Obj -3.5
Optimal - objective value -3.5
Optimal objective -3.5 - 7 iterations time 0.002
Option for printingOptions changed from normal to all
To

In [24]:
import pulp
import numpy as np
import pandas as pd

# Example input and output data
inputs = np.array([
    [3, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])

# Number of DMUs, inputs, and outputs
n_dmus = inputs.shape[0]
n_inputs = inputs.shape[1]
n_outputs = outputs.shape[1]

# Indices for weighted and unweighted inputs and outputs
weighted_input_indices = [0, 2]  # Inputs 0 and 2 are weighted
unweighted_input_indices = [i for i in range(n_inputs) if i not in weighted_input_indices]

weighted_output_indices = [1]  # Output 1 is weighted
unweighted_output_indices = [r for r in range(n_outputs) if r not in weighted_output_indices]

# Weights for weighted inputs and outputs
input_weights = {0: 2, 2: 3}  # Input 0 weight = 2.5, Input 2 weight = 1.5
output_weights = {1: 1.5}  # Output 1 weight = 2.0

# Small epsilon for non-negativity constraints
epsilon = 1e-6

# Store results for each DMU
all_results = []

# Loop through each DMU and solve the M12 model
for dmu_0 in range(n_dmus):
    # Create an LP problem for the DMU
    prob = pulp.LpProblem(f"M12_Model_DMU_{dmu_0}", pulp.LpMaximize)

    # Decision variables for unweighted inputs and outputs
    phi = [pulp.LpVariable(f"phi_{i}_DMU_{dmu_0}", lowBound=epsilon) for i in unweighted_input_indices]
    omega = [pulp.LpVariable(f"omega_{r}_DMU_{dmu_0}", lowBound=epsilon) for r in unweighted_output_indices]

    # Decision variables for weighted inputs and outputs
    phi_0 = [pulp.LpVariable(f"phi_0_{i}_DMU_{dmu_0}", lowBound=None) for i in weighted_input_indices]
    omega_0 = [pulp.LpVariable(f"omega_0_{r}_DMU_{dmu_0}", lowBound=None) for r in weighted_output_indices]

    # Objective function: Maximize the efficiency score
    prob += (
        pulp.lpSum([omega[unweighted_output_indices.index(r)] * outputs[dmu_0, r] for r in unweighted_output_indices]) +
        pulp.lpSum([omega_0[weighted_output_indices.index(r)] * outputs[dmu_0, r] for r in weighted_output_indices]) -
        pulp.lpSum([phi[unweighted_input_indices.index(i)] * inputs[dmu_0, i] for i in unweighted_input_indices]) -
        pulp.lpSum([phi_0[weighted_input_indices.index(i)] * inputs[dmu_0, i] for i in weighted_input_indices]) +
        pulp.lpSum([input_weights[i] for i in weighted_input_indices]) -
        pulp.lpSum([output_weights[r] for r in weighted_output_indices])
    ), f"Objective_DMU_{dmu_0}"

    # Constraints for each peer DMU
    for j in range(n_dmus):
        prob += (
            pulp.lpSum([omega[unweighted_output_indices.index(r)] * outputs[j, r] for r in unweighted_output_indices]) +
            pulp.lpSum([omega_0[weighted_output_indices.index(r)] * outputs[j, r] for r in weighted_output_indices]) -
            pulp.lpSum([phi[unweighted_input_indices.index(i)] * inputs[j, i] for i in unweighted_input_indices]) -
            pulp.lpSum([phi_0[weighted_input_indices.index(i)] * inputs[j, i] for i in weighted_input_indices])
            <= 0
        ), f"Peer_Constraint_{j}_DMU_{dmu_0}"

    # Constraints for weighted outputs
    for r in weighted_output_indices:
        prob += omega_0[weighted_output_indices.index(r)] * outputs[dmu_0, r] >= output_weights[r], \
                f"Weighted_Output_Constraint_{r}_DMU_{dmu_0}"

    # Constraints for weighted inputs
    for i in weighted_input_indices:
        prob += phi_0[weighted_input_indices.index(i)] * inputs[dmu_0, i] >= input_weights[i], \
                f"Weighted_Input_Constraint_{i}_DMU_{dmu_0}"

    # Solve the LP problem for the current DMU
    prob.solve()
    # Calculate efficiency score using the optimized weights
    numerator = (
        sum(pulp.value(omega[unweighted_output_indices.index(r)]) * outputs[dmu_0, r] for r in unweighted_output_indices) +
        sum(pulp.value(omega_0[weighted_output_indices.index(r)]) * outputs[dmu_0, r] for r in weighted_output_indices)
    )
    denominator = (
        sum(pulp.value(phi[unweighted_input_indices.index(i)]) * inputs[dmu_0, i] for i in unweighted_input_indices) +
        sum(pulp.value(phi_0[weighted_input_indices.index(i)]) * inputs[dmu_0, i] for i in weighted_input_indices)
    )
    efficiency_score = numerator / denominator if denominator > 0 else 0
    # Extract the results for the DMU
    phi_values = [pulp.value(p) for p in phi]
    phi_0_values = [pulp.value(p) for p in phi_0]
    omega_values = [pulp.value(o) for o in omega]
    omega_0_values = [pulp.value(o) for o in omega_0]

    # Store results in a structured way
    result = {
        "DMU": dmu_0,
        "Efficiency Score": efficiency_score,
        "Numerator (Outputs)": numerator,
        "Denominator (Inputs)": denominator,
        "phi (Unweighted Inputs)": phi_values,
        "phi_0 (Weighted Inputs)": phi_0_values,
        "omega (Unweighted Outputs)": omega_values,
        "omega_0 (Weighted Outputs)": omega_0_values
    }
    all_results.append(result)

# Convert results to a DataFrame and display them
df_results = pd.DataFrame(all_results)
print(df_results)


python(82748) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/84231ac7f83c4b1bba31d2573cf4da62-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/84231ac7f83c4b1bba31d2573cf4da62-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 47 RHS
At line 56 BOUNDS
At line 62 ENDATA
Problem MODEL has 8 rows, 5 columns and 28 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (-3) rows, 5 (0) columns and 25 (-3) elements
0  Obj -3.499994 Dual inf 16.999998 (2)
0  Obj -3.499994 Dual inf 16.999998 (2)
3  Obj -4.4408921e-16
Optimal - objective value 0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 3 iterations time 0.002, Presolve 0.00


python(82749) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82750) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82751) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(82752) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
